# Работа со слоем Silver

# Предобработка

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",             # S3
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/opt/spark/external-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

25/11/16 18:35:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# Треним JOIN

In [2]:
df_norm = spark.read.parquet("s3a://silver/nyc-taxi-data-norm/yellow_tripdata_2025-01")

25/11/16 18:37:13 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [10]:
list_vendor = (spark.read
      .format("jdbc")
      .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
      .option("driver", "org.postgresql.Driver")
      .option("user", "airflow")
      .option("password", "airflow")
      .option("dbtable", "nyc_taxi.list_vendor")
      .load())

list_ratecode = (spark.read
      .format("jdbc")
      .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
      .option("driver", "org.postgresql.Driver")
      .option("user", "airflow")
      .option("password", "airflow")
      .option("dbtable", "nyc_taxi.list_ratecode")
      .load())

list_payment = (spark.read
      .format("jdbc")
      .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
      .option("driver", "org.postgresql.Driver")
      .option("user", "airflow")
      .option("password", "airflow")
      .option("dbtable", "nyc_taxi.list_payment")
      .load())

list_taxi_zone = (spark.read
      .format("jdbc")
      .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
      .option("driver", "org.postgresql.Driver")
      .option("user", "airflow")
      .option("password", "airflow")
      .option("dbtable", "nyc_taxi.list_taxi_zone")
      .load())

df = (df_norm
      .join(list_vendor,   "vendorid",     "left")
      .join(list_ratecode, "ratecodeid",   "left")
      .join(list_payment,  "payment_type", "left")
      .join(list_taxi_zone.alias("pul_zones"), df_norm.pulocationid == F.col("pul_zones.locationid"), "left")
      .join(list_taxi_zone.alias("dol_zones"), df_norm.dolocationid == F.col("dol_zones.locationid"), "left")
      .select(
          df_norm["*"], 
          list_vendor.vendor_name,
          list_ratecode.ratecode_name,
          list_payment.payment_name,
          F.col("pul_zones.borough").alias("pickup_borough"),
          F.col("pul_zones.zone").alias("pickup_zone"),
          F.col("dol_zones.borough").alias("dropoff_borough"),
          F.col("dol_zones.zone").alias("dropoff_zone"),
             )
     )

In [11]:
print(df.columns)

['vendorid', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'ratecodeid', 'store_and_fwd_flag', 'pulocationid', 'dolocationid', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee', 'cbd_congestion_fee', 'vendor_name', 'ratecode_name', 'payment_name', 'pickup_borough', 'pickup_zone', 'dropoff_borough', 'dropoff_zone']


In [14]:
# Базовая информация
print("Количество строк и столбцов:", (df.count(), len(df.columns)))

print("Схема:")
df.select(['vendorid', 'ratecodeid', 'pulocationid', 'dolocationid', 
                 'payment_type', 'vendor_name', 'ratecode_name', 
                 'payment_name', 'pickup_borough', 'dropoff_borough',
                 'pickup_zone', 'dropoff_zone'
                ]).printSchema()

print("Предпросмотр")
print(df.select(['vendorid', 'ratecodeid', 'pulocationid', 'dolocationid', 
                 'payment_type', 'vendor_name', 'ratecode_name', 
                 'payment_name', 'pickup_borough', 'dropoff_borough',
                 'pickup_zone', 'dropoff_zone'
                ])
      .show(5))

Количество строк и столбцов: (3475226, 27)
Схема:
root
 |-- vendorid: integer (nullable = true)
 |-- ratecodeid: integer (nullable = true)
 |-- pulocationid: integer (nullable = true)
 |-- dolocationid: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- vendor_name: string (nullable = true)
 |-- ratecode_name: string (nullable = true)
 |-- payment_name: string (nullable = true)
 |-- pickup_borough: string (nullable = true)
 |-- dropoff_borough: string (nullable = true)
 |-- pickup_zone: string (nullable = true)
 |-- dropoff_zone: string (nullable = true)

Предпросмотр
+--------+----------+------------+------------+------------+--------------------+-------------+------------+--------------+---------------+--------------------+--------------------+
|vendorid|ratecodeid|pulocationid|dolocationid|payment_type|         vendor_name|ratecode_name|payment_name|pickup_borough|dropoff_borough|         pickup_zone|        dropoff_zone|
+--------+----------+------------+--

In [ ]:
import time

In [ ]:
start_time = time.time()

(df.write.format("jdbc")
         .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
         .option("driver", "org.postgresql.Driver")
         .option("user", "airflow")
         .option("password", "airflow")
         .option("dbtable", "nyc_taxi.list_taxi_zone")
         .option("batchsize", 10000)
         .mode("overwrite")
         .save())


execution_time = time.time() - start_time
print(f"⏱️  Записали за: {execution_time:.2f} секунд ({execution_time / 60:.2f} минут)")


In [ ]:
# Обработка данных из norm в eda
eda_incremental_nyc_taxi_files(
    input_bucket='silver', 
    input_prefix='nyc-taxi-data-norm/',
    output_bucket='silver', 
    output_prefix='nyc-taxi-data-eda/'
)

In [29]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/opt/spark/external-jars/hadoop-aws-3.3.4.jar",             # S3
    "/opt/spark/external-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/opt/spark/external-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/opt/spark/external-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

In [13]:
df = spark.read.parquet("s3a://bronze/nyc-taxi-data/yellow_tripdata_2025-07.parquet") #yellow_tripdata_2025-09/")

In [25]:
df = spark.read.parquet("s3a://silver/nyc-taxi-data-norm/yellow_tripdata_2025-07/") #yellow_tripdata_2025-09/")

In [68]:
df = spark.read.parquet("s3a://silver/nyc-taxi-data-eda/yellow_tripdata_2025-09") #yellow_tripdata_2025-09/")

In [41]:
df = spark.read.parquet("s3a://silver/nyc-taxi-data-eda/*") #yellow_tripdata_2025-09/")

In [27]:
spark.stop()

In [69]:
df.columns

['vendorid',
 'vendor_name',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'ratecodeid',
 'ratecode_name',
 'pulocationid',
 'pickup_borough',
 'pickup_zone',
 'dolocationid',
 'dropoff_borough',
 'dropoff_zone',
 'payment_type',
 'payment_name',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'airport_fee',
 'trip_duration_minutes',
 'date_month',
 'year',
 'month',
 'day',
 'day_of_week',
 'hour',
 'is_weekend',
 'time_of_day',
 'is_rush_hour',
 'avg_speed_kmh',
 'tip_ratio',
 'has_tip',
 'revenue_per_minute']

In [71]:
df.filter("month = 9").groupBy(F.year("tpep_pickup_datetime"), F.month("tpep_pickup_datetime")).count().show(50)

[Stage 134:==================================================>     (9 + 1) / 10]

+--------------------------+---------------------------+-------+
|year(tpep_pickup_datetime)|month(tpep_pickup_datetime)|  count|
+--------------------------+---------------------------+-------+
|                      2025|                          9|3784662|
+--------------------------+---------------------------+-------+



In [42]:
# Получаем список столбцов
cols = df.columns

# Считаем NULL по каждому столбцу
null_counts = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in cols])

# Получаем результат как словарь или строку
null_dict = null_counts.collect()[0].asDict()

for i in null_dict.items():
    print(i)

[Stage 80:==================================================>     (44 + 5) / 49]

('vendorid', 0)
('vendor_name', 0)
('tpep_pickup_datetime', 0)
('tpep_dropoff_datetime', 0)
('passenger_count', 0)
('trip_distance', 0)
('ratecodeid', 0)
('ratecode_name', 0)
('pulocationid', 0)
('pickup_borough', 0)
('pickup_zone', 0)
('dolocationid', 0)
('dropoff_borough', 0)
('dropoff_zone', 0)
('payment_type', 0)
('payment_name', 0)
('fare_amount', 0)
('extra', 0)
('mta_tax', 0)
('tip_amount', 0)
('tolls_amount', 0)
('improvement_surcharge', 0)
('total_amount', 0)
('congestion_surcharge', 0)
('airport_fee', 0)
('trip_duration_minutes', 0)
('date_month', 0)
('year', 0)
('month', 0)
('day', 0)
('day_of_week', 0)
('hour', 0)
('is_weekend', 0)
('time_of_day', 0)
('is_rush_hour', 0)
('avg_speed_kmh', 0)
('tip_ratio', 0)
('has_tip', 0)
('revenue_per_minute', 0)


In [43]:
null_rows = (df.select(
    [
    'year',
    'month',
    'vendorid', 
    'vendor_name',])
             .filter(F.col("vendor_name").isNull())
             .groupBy('year',
                      'month',
                      'vendorid', 
                      'vendor_name',).count()
            )
null_rows.show(truncate=False)

+----+-----+--------+-----------+-----+
|year|month|vendorid|vendor_name|count|
+----+-----+--------+-----------+-----+
+----+-----+--------+-----------+-----+



In [16]:
df.groupBy("date_month", "year", "month").count().orderBy("year", "month").show(50)

[Stage 17:=====================================================>  (45 + 2) / 47]

+-------------------+----+-----+-------+
|         date_month|year|month|  count|
+-------------------+----+-----+-------+
|2022-01-01 00:00:00|2022|    1|2405105|
|2022-02-01 00:00:00|2022|    2|2911596|
|2022-03-01 00:00:00|2022|    3|3540060|
|2022-04-01 00:00:00|2022|    4|3509368|
|2022-05-01 00:00:00|2022|    5|3487114|
|2022-06-01 00:00:00|2022|    6|3447089|
|2022-07-01 00:00:00|2022|    7|3081872|
|2022-08-01 00:00:00|2022|    8|3056951|
|2022-09-01 00:00:00|2022|    9|3080827|
|2022-10-01 00:00:00|2022|   10|3556349|
|2022-11-01 00:00:00|2022|   11|3140479|
|2022-12-01 00:00:00|2022|   12|3277500|
|2023-01-01 00:00:00|2023|    1|2971693|
|2023-02-01 00:00:00|2023|    2|2823988|
|2023-03-01 00:00:00|2023|    3|3293502|
|2023-04-01 00:00:00|2023|    4|3184656|
|2023-05-01 00:00:00|2023|    5|3396540|
|2023-06-01 00:00:00|2023|    6|3192332|
|2023-07-01 00:00:00|2023|    7|2795870|
|2023-08-01 00:00:00|2023|    8|2706311|
|2023-09-01 00:00:00|2023|    9|2681346|
|2023-10-01 00:0

In [8]:
df.printSchema()

root
 |-- vendorid: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- pulocationid: integer (nullable = true)
 |-- dolocationid: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



# Агрегаты

In [ ]:
df.groupBy("hour").agg(
    F.count("*").alias("total_trips"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability"),
    F.sum("total_amount").alias("total_revenue")
).orderBy("hour").show(30)

In [ ]:
df.groupBy("day_of_week", "is_weekend").agg(
    F.count("*").alias("total_trips"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("passenger_count").alias("avg_passengers")
).orderBy("day_of_week").show()

In [ ]:
df.groupBy("time_of_day").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
).show()

In [ ]:
df.filter(F.col("avg_speed_kmh") > 0).groupBy(
    F.when(F.col("avg_speed_kmh") < 10, "0-10 kmh")
     .when(F.col("avg_speed_kmh") < 20, "10-20 kmh") 
     .when(F.col("avg_speed_kmh") < 30, "20-30 kmh")
     .otherwise("30+ kmh").alias("speed_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("revenue_per_minute").alias("avg_revenue_per_min")
).show()

In [ ]:
df.groupBy(
    F.when(F.col("trip_distance") < 1, "0-1 mi")
     .when(F.col("trip_distance") < 3, "1-3 mi")
     .when(F.col("trip_distance") < 5, "3-5 mi")
     .when(F.col("trip_distance") < 10, "5-10 mi")
     .otherwise("10+ mi").alias("distance_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
).show()

In [ ]:
df.groupBy("payment_type").agg(
    F.count("*").alias("transaction_count"),
    F.avg("total_amount").alias("avg_amount"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability")
).show()

Отличная структура! Вот комплексные агрегаты для анализа и дашбордов, сгруппированные по ключевым темам.

### 1. Временные агрегаты (по часам, дням недели, weekend/weekday)

```python
# Агрегаты по часам
time_hourly_agg = df.groupBy("hour").agg(
    F.count("*").alias("total_trips"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability"),
    F.sum("total_amount").alias("total_revenue")
).orderBy("hour")

# Агрегаты по дням недели
daily_agg = df.groupBy("day_of_week", "is_weekend").agg(
    F.count("*").alias("total_trips"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("passenger_count").alias("avg_passengers")
).orderBy("day_of_week")
```

### 2. Географические агрегаты (по зонам посадки/высадки)

```python
# Топ зон посадки
top_pickup_locations = df.groupBy("pulocationid").agg(
    F.count("*").alias("pickup_count"),
    F.avg("total_amount").alias("avg_fare_from_zone"),
    F.avg("trip_duration_minutes").alias("avg_duration_from_zone"),
    F.avg("tip_ratio").alias("avg_tip_ratio_from_zone")
).orderBy(F.desc("pickup_count"))

# Топ маршрутов (путь из A в B)
top_routes = df.groupBy("pulocationid", "dolocationid").agg(
    F.count("*").alias("route_popularity"),
    F.avg("trip_distance").alias("avg_route_distance"),
    F.avg("total_amount").alias("avg_route_revenue"),
    F.avg("trip_duration_minutes").alias("avg_route_duration")
).orderBy(F.desc("route_popularity"))
```

### 3. Агрегаты по типам поездок и тарифам

```python
# По ratecodeid (тип поездки)
ratecode_agg = df.groupBy("ratecodeid").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_amount"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("trip_duration_minutes").alias("avg_duration")
)

# Анализ по времени суток
time_of_day_agg = df.groupBy("time_of_day").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
)
```

### 4. Анализ эффективности и доходности

```python
# Агрегаты по скорости и дистанции
speed_analysis = df.filter(F.col("avg_speed_kmh") > 0).groupBy(
    F.when(F.col("avg_speed_kmh") < 10, "0-10 kmh")
     .when(F.col("avg_speed_kmh") < 20, "10-20 kmh") 
     .when(F.col("avg_speed_kmh") < 30, "20-30 kmh")
     .otherwise("30+ kmh").alias("speed_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("revenue_per_minute").alias("avg_revenue_per_min")
)

# Анализ по расстоянию
distance_analysis = df.groupBy(
    F.when(F.col("trip_distance") < 1, "0-1 mi")
     .when(F.col("trip_distance") < 3, "1-3 mi")
     .when(F.col("trip_distance") < 5, "3-5 mi")
     .when(F.col("trip_distance") < 10, "5-10 mi")
     .otherwise("10+ mi").alias("distance_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
)
```

### 5. Анализ чаевых и платежей

```python
# Анализ чаевых по времени и типу оплаты
tip_analysis = df.groupBy("payment_type", "time_of_day").agg(
    F.count("*").alias("total_transactions"),
    F.sum("has_tip").alias("tipped_transactions"),
    F.avg("tip_amount").alias("avg_tip_amount"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    (F.sum("has_tip") / F.count("*")).alias("tip_frequency")
)

# Распределение по типам платежей
payment_analysis = df.groupBy("payment_type").agg(
    F.count("*").alias("transaction_count"),
    F.avg("total_amount").alias("avg_amount"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability")
)
```

### 6. Агрегаты по водителям/провайдерам

```python
# Сравнение вендоров
vendor_analysis = df.groupBy("vendorid").agg(
    F.count("*").alias("total_trips"),
    F.avg("total_amount").alias("avg_revenue_per_trip"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("passenger_count").alias("avg_passengers"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.sum("total_amount").alias("total_revenue")
)
```

### 7. Комплексные кросс-агрегаты

```python
# Время суток + день недели
cross_time_analysis = df.groupBy("time_of_day", "day_of_week").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("revenue_per_minute").alias("efficiency")
)

# География + время
location_time_analysis = df.groupBy("pulocationid", "time_of_day").agg(
    F.count("*").alias("pickup_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration")
)
```

### Рекомендации для дашборда:

**Главные метрики KPI:**
- Общее количество поездок
- Общая выручка
- Средний чек
- Процент поездок с чаевыми
- Средняя скорость

**Основные графики:**
1. **Тепловая карта:** Часы дня × Дни недели (интенсивность поездок)
2. **График линии:** Динамика поездок по часам
3. **Bar chart:** Топ-10 зон посадки
4. **Scatter plot:** Связь расстояния и стоимости
5. **Pie chart:** Распределение по времени суток
6. **Histogram:** Распределение чаевых

Все эти агрегаты можно вычислять инкрементально и обновлять в дашборде при добавлении новых данных.

In [51]:
df.select(["year", "month"]).distinct().count()

46

In [55]:
df.filter(F.col("payment_name") == 'Cash').groupBy("year","month").count().orderBy("year","month").show(3)

+----+-----+------+
|year|month| count|
+----+-----+------+
|2022|    1|478950|
|2022|    2|536922|
|2022|    3|669612|
+----+-----+------+
only showing top 3 rows



In [57]:
df.filter("payment_name = 'Cash'").groupBy("year","month").count().orderBy("year","month").show(3)

+----+-----+------+
|year|month| count|
+----+-----+------+
|2022|    1|478950|
|2022|    2|536922|
|2022|    3|669612|
+----+-----+------+
only showing top 3 rows



In [59]:
df_agg = df.groupBy(
    "date_month", "year", "month", "day_of_week", "time_of_day", 
    "pulocationid", "pickup_borough", "pickup_zone",
    "payment_type", "payment_name"
).agg(
    F.count("*").alias("trip_count"),
    F.sum("total_amount").alias("total_revenue"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability"),
    F.avg("passenger_count").alias("avg_passengers"),
    F.avg("revenue_per_minute").alias("avg_efficiency")
)

In [60]:
df_agg.count()

781070

In [67]:
driver_efficiency = df \
    .withColumn("revenue_per_hour", 
               F.col("total_amount") / 
               ((unix_timestamp("tpep_dropoff_datetime") - 
                 unix_timestamp("tpep_pickup_datetime")) / 3600)) \
    .withColumn("hour", hour("tpep_pickup_datetime")) \
    .withColumn("day_of_week", dayofweek("tpep_pickup_datetime")) \
    .groupBy("day_of_week", "hour") \
    .agg(avg("revenue_per_hour").alias("avg_revenue_per_hour"))

NameError: name 'unix_timestamp' is not defined

In [61]:
df_agg.show(5)

[Stage 124:====================================================>  (47 + 2) / 49]

+-------------------+----+-----+-----------+-----------+------------+--------------+--------------------+------------+------------+----------+------------------+------------------+------------------+------------------+------------------+-------------------+---------------+------------------+------------------+
|         date_month|year|month|day_of_week|time_of_day|pulocationid|pickup_borough|         pickup_zone|payment_type|payment_name|trip_count|     total_revenue|       avg_revenue|      avg_duration|      avg_distance|         avg_speed|      avg_tip_ratio|tip_probability|    avg_passengers|    avg_efficiency|
+-------------------+----+-----+-----------+-----------+------------+--------------+--------------------+------------+------------+----------+------------------+------------------+------------------+------------------+------------------+-------------------+---------------+------------------+------------------+
|2022-03-01 00:00:00|2022|    3|          3|       Утро|        

In [72]:
(df.filter("month = 9").write.format("jdbc")
         .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
         .option("driver", "org.postgresql.Driver")
         .option("user", "airflow")
         .option("password", "airflow")
         .option("dbtable", "nyc_taxi.yellow_tripdata_2025_09_eda")
         .option("batchsize", 10000)
         .mode("overwrite")
         .save())


## **Варианты графиков для дашборда:**

### **1. Временные тренды**
- **Линейный график:** Динамика поездок/выручки по месяцам (фильтр: конкретная зона или "все")
- **Heatmap:** Дни недели × Время суток (интенсивность поездок)

### **2. Географический анализ**  
- **Карта:** Тепловая карта по зонам (количество поездок/средний чек)
- **Bar chart:** Топ-10 зон по выручке/поездкам

### **3. Сравнительный анализ**
- **Scatter plot:** Эффективность (доход/минуту) vs Чаевые по зонам
- **Grouped bar:** Сравнение метрик по времени суток (утро/день/вечер/ночь)

### **4. Детализация по фильтрам**
- **Все графики** должны обновляться при выборе:
  - Год/Месяц
  - День недели 
  - Время суток
  - Зона посадки

**Преимущества подхода:**
- ✅ Одна таблица на все случаи
- ✅ Быстрые отклики на любые фильтры
- ✅ Возможность drill-down от общего к частному
- ✅ Укладываемся в 400k строк

Что скажете? Такой подход подходит? Какие метрики или разрезы хотите добавить/убрать?

In [ ]:
spark.stop()